<a href="https://colab.research.google.com/github/cs229-mind/mind/blob/main/NRMS_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NRMS: Neural News Recommendation with Multi-Head Self-Attention
The following documents are copied from MIND public document.

[NRMS](https://wuch15.github.io/paper/EMNLP2019-NRMS.pdf) is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user 
representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to tranform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Self Defined Module

In [ ]:

class AdditiveAttention(nn.Module):
    ''' AttentionPooling used to weighted aggregate news vectors
    Arg: 
        d_h: the last dimension of input
    '''
    def __init__(self, d_h, hidden_size=200):
        super(AdditiveAttention, self).__init__()
        self.att_fc1 = nn.Linear(d_h, hidden_size)
        self.att_fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x, attn_mask=None):
        """
        Args:
            x: batch_size, candidate_size, candidate_vector_dim
            attn_mask: batch_size, candidate_size
        Returns:
            (shape) batch_size, candidate_vector_dim
        """
        bz = x.shape[0]
        e = torch.tanh(self.att_fc1(x))
        #e = nn.Tanh()(e)
        alpha = self.att_fc2(e)

        alpha = torch.exp(alpha)
        if attn_mask is not None:
            alpha = alpha * attn_mask.unsqueeze(2)
        alpha = alpha / (torch.sum(alpha, dim=1, keepdim=True) + 1e-8)

        x = torch.bmm(x.permute(0, 2, 1), alpha)
        x = torch.reshape(x, (bz, -1))  # (bz, 400)
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask=None):
        #       [bz, 20, seq_len, 20] x [bz, 20, 20, seq_len] -> [bz, 20, seq_len, seq_len]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        scores = torch.exp(scores)
        if attn_mask is not None:
            scores = scores * attn_mask
        attn = scores / (torch.sum(scores, dim=-1, keepdim=True) + 1e-8)

        #       [bz, 20, seq_len, seq_len] x [bz, 20, seq_len, 20] -> [bz, 20, seq_len, 20]
        context = torch.matmul(attn, V)
        return context, attn


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k, d_v, enable_gpu):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model  # 300
        self.n_heads = n_heads  # 20
        self.d_k = d_k  # 20
        self.d_v = d_v  # 20
        self.enable_gpu = enable_gpu

        self.W_Q = nn.Linear(d_model, d_k * n_heads)  # 300, 400
        self.W_K = nn.Linear(d_model, d_k * n_heads)  # 300, 400
        self.W_V = nn.Linear(d_model, d_v * n_heads)  # 300, 400

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)

    def forward(self, Q, K, V, mask=None):
        #       Q, K, V: [bz, seq_len, 300] -> W -> [bz, seq_len, 400]-> q_s: [bz, 20, seq_len, 20]
        batch_size, seq_len, _ = Q.shape

        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads,
                               self.d_k).transpose(1, 2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads,
                               self.d_k).transpose(1, 2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads,
                               self.d_v).transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1).expand(batch_size, seq_len, seq_len) #  [bz, seq_len, seq_len]
            mask = mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [bz, 20, seq_len, seq_len]

        context, attn = ScaledDotProductAttention(self.d_k)(
            q_s, k_s, v_s, mask)  # [bz, 20, seq_len, 20]
        context = context.transpose(1, 2).contiguous().view(
            batch_size, -1, self.n_heads * self.d_v)  # [bz, seq_len, 400]
        #         output = self.fc(context)
        return context  #self.layer_norm(output + residual)


class TextEmbedding(torch.nn.Module):
    def __init__(self,
                 bert_model,
                 dropout=0.0,
                 layers=None,
                 enable_gpu=True):
        super(TextEmbedding, self).__init__()
        #self.word_embedding = word_embedding
        self.bert_model = bert_model  ## output embeddings from pretrained model
        self.layers = [-4, -3, -2, -1] if layers is None else layers ## Use last 4 layers hidden state average to represent the embedding
        self.Dropout = torch.nn.Dropout(p=dropout)

    def forward(self, text, mask=None):
        """
        Args:
            text: Tensor(batch_size) * num_words_text * embedding_dim
        Returns:
            (shape) batch_size, word_embedding_dim
        """
        # batch_size, num_words_text
        batch_size, num_words = text.shape
        num_words = num_words // 3
        text_ids = torch.narrow(text, 1, 0, num_words)
        text_type = torch.narrow(text, 1, num_words, num_words)
        text_attmask = torch.narrow(text, 1, num_words*2, num_words)
        states = self.bert_model(text_ids, text_type, text_attmask).hidden_states
        word_emb = states[-1] #torch.stack([states[i] for i in self.layers]).sum(0).squeeze()
        return self.Dropout(word_emb)


# class ElementEncoder(torch.nn.Module):
#     def __init__(self, num_elements, embedding_dim, enable_gpu=True):
#         super(ElementEncoder, self).__init__()
#         self.enable_gpu = enable_gpu
#         self.embedding = nn.Embedding(num_elements,
#                                       embedding_dim,
#                                       padding_idx=0)
#
#     def forward(self, element):
#         # batch_size, embedding_dim
#         element_vector = self.embedding(
#             (element.cuda() if self.enable_gpu else element).long())
#         return element_vector


class NewsEncoder(torch.nn.Module):
    def __init__(self, args, bert_model, category_dict_size,
                 domain_dict_size, subcategory_dict_size, enable_gpu=True):
        super(NewsEncoder, self).__init__()
        self.args = args
        self.attributes2length = {
            'title': args.num_words_title * 3,
            'abstract': args.num_words_abstract * 3,
            'body': args.num_words_body * 3,
            'category': 1,
            'domain': 1,
            'subcategory': 1
        }
        for key in list(self.attributes2length.keys()):
            if key not in args.news_attributes:
                self.attributes2length[key] = 0

        self.attributes2start = {
            key: sum(
                list(self.attributes2length.values())
                [:list(self.attributes2length.keys()).index(key)])
            for key in self.attributes2length.keys()
        }

        assert len(args.news_attributes) > 0
        text_encoders_candidates = ['title', 'abstract']

        self.text_encoders = nn.ModuleDict({
            'title':
            TextEmbedding(bert_model, dropout=args.drop_rate)
        })

        self.newsname=[name for name in sorted(list(set(args.news_attributes) & set(text_encoders_candidates)))]

        head_dim = args.news_dim // args.num_attention_heads

        self.Multihead = MultiHeadAttention(args.word_emb_size,
                                            args.num_attention_heads, head_dim,
                                            head_dim, enable_gpu)  # args.news_dim = 400 = 20 * num_attention_heads
        self.additiveatt = AdditiveAttention(args.news_dim, hidden_size=200)
        self.Dropout = torch.nn.Dropout(args.drop_rate)

    def forward(self, news):
        """
        Args:
        Returns:
            (shape) batch_size, news_dim
        """
        text_vectors = [
            self.text_encoders['title'](
                torch.narrow(news, 1, self.attributes2start[name],
                             self.attributes2length[name]))
            for name in self.newsname
        ]

        # batch_size, news_dim
        input = text_vectors[0]
        output = self.Multihead(input, input, input) # N * L * E
        output = self.Dropout(output)
        output = self.additiveatt(output) # N * E
        return output

class UserEncoder(torch.nn.Module):
    def __init__(self, args, enable_gpu=True):
        super(UserEncoder, self).__init__()
        self.args = args
        head_dim = args.news_dim // args.num_attention_heads

        self.Multihead = MultiHeadAttention(args.news_dim,
                                            args.num_attention_heads, head_dim,
                                            head_dim, enable_gpu)
        self.additiveatt = AdditiveAttention(args.news_dim, hidden_size=200)
        self.Dropout = torch.nn.Dropout(args.drop_rate)


    def forward(self, log_vec, log_mask=None):
        """
        Returns:
            (shape) batch_size,  news_dim
        """
        output = self.Multihead(log_vec, log_vec, log_vec)  # N * L * E
        output = self.Dropout(output)
        output = self.additiveatt(output)  # N * E
        return output
class TwoTowerModel(torch.nn.Module):
    """
    UniUM network.
    Input 1 + K candidate news and a list of user clicked news, produce the click probability.
    """
    def __init__(self,
                 args,
                 bert_model,
                 user_dict_size=0,
                 category_dict_size=0,
                 domain_dict_size=0,
                 subcategory_dict_size=0):
        super(TwoTowerModel, self).__init__()
        self.args = args


        self.news_encoder = NewsEncoder(args,
                                        bert_model,
                                        category_dict_size, 
                                        domain_dict_size,
                                        subcategory_dict_size)
        self.user_encoder = UserEncoder(args)

        self.criterion = nn.CrossEntropyLoss()

    def forward(self,
                input_ids,
                log_ids,
                log_mask,
                targets=None,
                compute_loss=True):
        """
        Returns:
          click_probability: batch_size, 1 + K
        """
        # input_ids: batch, 1+npratio, num_words
        ids_length = input_ids.size(2)
        input_ids = input_ids.view(-1, ids_length) # change to batch * (1+npratio), num_words
        news_vec = self.news_encoder(input_ids)
        news_vec = news_vec.view(-1, 1 + self.args.npratio, self.args.news_dim)

        # batch_size, news_dim
        log_ids = log_ids.view(-1, ids_length)
        log_vec = self.news_encoder(log_ids)
        log_vec = log_vec.view(-1, self.args.user_log_length,
                               self.args.news_dim)

        user_vector = self.user_encoder(log_vec, log_mask)

        # batch_size, 2
        score = torch.bmm(news_vec, user_vector.unsqueeze(-1)).squeeze(dim=-1)
        if compute_loss:
            loss = self.criterion(score, targets)
            return loss, score
        else:
            return score




## Global settings and imports

In [ ]:
#!ls gdrive/MyDrive
!pip install transformers horovod

In [ ]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# %cd /content
# !mkdir drive
# %cd drive
# !mkdir MyDrive
# %cd ..
# %cd ..
# !google-drive-ocamlfuse "/content/drive/MyDrive"
# import sys
# path = '/content/drive/MyDrive'
# sys.path.append(path)
#data_path = '/content/drive/MyDrive'

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')
import sys
#define util module path
path = '/content/gdrive/My Drive'
sys.path.append(path)
data_path = '/content/gdrive/My Drive'
# import sys
# import os
# os.getcwd()
%tensorflow_version 2.x
import os
import numpy as np
import zipfile
from tqdm import tqdm
#import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from NRMS.dataloader import DataLoaderTrain, DataLoaderTest
from torch.utils.data import Dataset, DataLoader
from NRMS.preprocess import read_news, read_news_bert, get_doc_input, get_doc_input_bert
from NRMS.model_bert import ModelBert
from NRMS.parameters import parse_args
import torch.optim as optim
from pathlib import Path
import NRMS.utils as utils
import os
import sys
import csv
import logging
import datetime
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
from torch import nn
from NRMS.nrms import TwoTowerModel as TwoTower
print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))
print("Pytorch version: {}".format(torch.__version__))



Mounted at /content/gdrive
System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 2.7.0
Pytorch version: 1.10.0+cu111


## Prepare parameters

In [ ]:
utils.setuplogger()
args = parse_args()
args.enable_hvd = False
args.batch_size = 32
args.news_dim = 400
args.word_emb_size = 768
data_path = '/content/gdrive/My Drive' # '/content/drive/MyDrive'
train_dir = os.path.join(data_path, 'MINDsmall_train')
train_news_file = os.path.join(train_dir, r'news.tsv')
train_behaviors_file = os.path.join(train_dir, r'behaviors.tsv')

valid_dir = os.path.join(data_path, 'MINDsmall_dev')
valid_news_file = os.path.join(valid_dir, r'news.tsv')
valid_behaviors_file = os.path.join(valid_dir, r'behaviors.tsv')

test_dir = os.path.join(data_path, 'MINDsmall_test')
test_news_file = os.path.join(test_dir, r'news.tsv')
test_behaviors_file = os.path.join(test_dir, r'behaviors.tsv')
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
config = AutoConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
bert_model = AutoModel.from_pretrained("bert-base-uncased",config=config)
bert_model = bert_model.cuda()

finetuneset={
'encoder.layer.10.attention.self.query.weight',
'encoder.layer.10.attention.self.query.bias',
'encoder.layer.10.attention.self.key.weight',
'encoder.layer.10.attention.self.key.bias',
'encoder.layer.10.attention.self.value.weight',
'encoder.layer.10.attention.self.value.bias',
'encoder.layer.10.attention.output.dense.weight',
'encoder.layer.10.attention.output.dense.bias',
'encoder.layer.10.attention.output.LayerNorm.weight',
'encoder.layer.10.attention.output.LayerNorm.bias',
'encoder.layer.10.intermediate.dense.weight',
'encoder.layer.10.intermediate.dense.bias',
'encoder.layer.10.output.dense.weight',
'encoder.layer.10.output.dense.bias',
'encoder.layer.10.output.LayerNorm.weight',
'encoder.layer.10.output.LayerNorm.bias',
'encoder.layer.11.attention.self.query.weight',
'encoder.layer.11.attention.self.query.bias',
'encoder.layer.11.attention.self.key.weight',
'encoder.layer.11.attention.self.key.bias',
'encoder.layer.11.attention.self.value.weight',
'encoder.layer.11.attention.self.value.bias',
'encoder.layer.11.attention.output.dense.weight',
'encoder.layer.11.attention.output.dense.bias',
'encoder.layer.11.attention.output.LayerNorm.weight',
'encoder.layer.11.attention.output.LayerNorm.bias',
'encoder.layer.11.intermediate.dense.weight',
'encoder.layer.11.intermediate.dense.bias',
'encoder.layer.11.output.dense.weight',
'encoder.layer.11.output.dense.bias',
'encoder.layer.11.output.LayerNorm.weight',
'encoder.layer.11.output.LayerNorm.bias',
'pooler.dense.weight',
'pooler.dense.bias',
'rel_pos_bias.weight',
'classifier.weight',
'classifier.bias'}
for name,param in bert_model.named_parameters():
    if name not in finetuneset:
        param.requires_grad = False


[INFO 2021-11-23 08:10:50,155] Namespace(batch_size=512, config_name='../Turing/unilm2-base-uncased-config.json', dataset='MIND', do_lower_case=True, drop_rate=0.2, embedding_source='random', enable_gpu=True, enable_hvd=True, epochs=4, f='/root/.local/share/jupyter/runtime/kernel-4dbea63a-6767-478a-9a52-15d882142a3f.json', filename_pat='behaviors*.tsv', filter_num=0, freeze_embedding=False, load_ckpt_name='epoch-1-40000.pt', log_steps=100, lr=0.0001, max_steps_per_epoch=200000, mode='test', model_dir='./model', model_name_or_path='../Turing/unilm2-base-uncased.bin', model_type='tnlrv3', news_attributes=['title'], news_dim=64, news_query_vector_dim=200, npratio=1, num_attention_heads=20, num_words_abstract=50, num_words_bing=26, num_words_body=50, num_words_title=24, num_words_uet=16, num_workers=2, padded_news_different_word_index=False, pretrain_news_encoder_path='.', process_bing=False, process_uet=False, root_data_dir='~/mind/', save_steps=500, shuffle_buffer_size=10000, test_dir='t

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Download and load data

In [ ]:
news, news_index, category_dict, domain_dict, subcategory_dict = read_news_bert(
    train_news_file, 
    args,
    tokenizer
)

news_title, news_title_type, news_title_attmask, \
news_abstract, news_abstract_type, news_abstract_attmask, \
news_body, news_body_type, news_body_attmask, \
news_category, news_domain, news_subcategory = get_doc_input_bert(
    news, news_index, category_dict, domain_dict, subcategory_dict, args)

word_dict = None
#args.enable_hvd = False
hvd_size, hvd_rank, hvd_local_rank = utils.init_hvd_cuda(
        args.enable_hvd, args.enable_gpu)
news_combined = np.concatenate([
        x for x in
        [news_title, news_title_type, news_title_attmask, \
            news_abstract, news_abstract_type, news_abstract_attmask, \
            news_body, news_body_type, news_body_attmask, \
            news_category, news_domain, news_subcategory]
        if x is not None], axis=1)
dataloader = DataLoaderTrain(
        news_index=news_index,
        news_combined=news_combined,
        word_dict=word_dict,
        data_dir=train_dir,
        filename_pat=args.filename_pat,
        args=args,
        worker_size=hvd_size,
        worker_rank=hvd_rank,
        cuda_device_idx=hvd_local_rank,
        enable_prefetch=True,
        enable_shuffle=True,
        enable_gpu=args.enable_gpu,
    )
    

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
51282it [00:07, 6572.43it/s]
100%|██████████| 51282/51282 [00:00<00:00, 87071.29it/s]


## Train Model

In [ ]:
#from NRMS.nrms import TwoTowerModel as TwoTower
model = TwoTower(args, bert_model, len(category_dict), len(domain_dict), len(subcategory_dict))
args.optimizer = 'Adam'
args.enable_lr_scheduler = False
if args.enable_gpu:
    model = model.cuda()
if args.optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
elif args.optimizer == 'AdamW':
    optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay, correct_bias=args.correct_bias)
else:
    optimizer = AdamW(model.parameters(), lr=args.lr)
if args.enable_lr_scheduler:
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=num_training_steps
    )
logging.info('Training...')
LOSS, ACC = [], []
args.epochs=1
args.max_steps_per_epoch = 1000
for ep in range(args.epochs):
    loss = 0.0
    accuracy = 0.0
    tqdm_util = tqdm(enumerate(dataloader))
    for cnt, (log_ids, log_mask, input_ids, targets) in tqdm_util:
        if cnt > args.max_steps_per_epoch:
            break

        if args.enable_gpu:
            log_ids = log_ids.cuda(non_blocking=True)
            log_mask = log_mask.cuda(non_blocking=True)
            input_ids = input_ids.cuda(non_blocking=True)
            #user_ids = user_ids.cuda(non_blocking=True)
            targets = targets.cuda(non_blocking=True)

        bz_loss, y_hat = model(input_ids, log_ids, log_mask, targets)
        # summary(model, [input_ids.shape, log_ids.shape, log_mask.shape, targets.shape], batch_size=16, device='cuda' if args.enable_gp else 'cpu')
        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()
        if args.enable_lr_scheduler:
            lr_scheduler.step()

        loss += (bz_loss.data.float() - loss) / (cnt + 1)
        accuracy += (utils.acc(targets, y_hat) - accuracy) / (cnt + 1)
        if cnt % args.log_steps == 0:
            LOSS.append(loss.data)
            ACC.append(accuracy)
            #logging.info(
            tqdm_util.set_description(
                '[{}] Ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(
                    hvd_rank, cnt * args.batch_size, loss.data,
                    accuracy))

        # save model minibatch
        #logging.info('[{}] Ed: {} {} {}'.format(hvd_rank, cnt, args.save_steps, cnt % args.save_steps))
    logging.info('epoch: {} loss: {:.5f} accuracy {:.5f}'.format(ep + 1, loss, accuracy))


[INFO 2021-11-23 08:11:04,362] Training...
[INFO 2021-11-23 08:11:04,365] DataLoader __iter__()
get_files: /content/gdrive/My Drive/MINDsmall_train, behaviors*.tsv

0it [00:00, ?it/s]


files: ['/content/gdrive/My Drive/MINDsmall_train/behaviors.tsv']
[INFO 2021-11-23 08:11:04,377] worker_rank:0, worker_size:1, shuffle:True, seed:0, directory:/content/gdrive/My Drive/MINDsmall_train, files:['/content/gdrive/My Drive/MINDsmall_train/behaviors.tsv']
[INFO 2021-11-23 08:11:04,380] data_paths: ['/content/gdrive/My Drive/MINDsmall_train/behaviors.tsv']


/content/gdrive/My Drive/mind_model/dataloader.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  user_feature_batch = torch.LongTensor(user_feature_batch).cuda()
[0] Ed: 32000, train_loss: 0.68936, acc: 0.60568: : 1001it [25:28,  1.53s/it]

[INFO 2021-11-23 08:36:32,669] epoch: 1 loss: 0.68936 accuracy 0.60568


In [ ]:
#args.log_steps

In [ ]:
#args.max_steps_per_epoch
news, news_index, category_dict, domain_dict, subcategory_dict = read_news_bert(
    valid_news_file, 
    args,
    tokenizer
)

news_title, news_title_type, news_title_attmask, \
news_abstract, news_abstract_type, news_abstract_attmask, \
news_body, news_body_type, news_body_attmask, \
news_category, news_domain, news_subcategory = get_doc_input_bert(
    news, news_index, category_dict, domain_dict, subcategory_dict, args)

word_dict = None
#args.enable_hvd = False
hvd_size, hvd_rank, hvd_local_rank = utils.init_hvd_cuda(
        args.enable_hvd, args.enable_gpu)
news_combined = np.concatenate([
        x for x in
        [news_title, news_title_type, news_title_attmask, \
            news_abstract, news_abstract_type, news_abstract_attmask, \
            news_body, news_body_type, news_body_attmask, \
            news_category, news_domain, news_subcategory]
        if x is not None], axis=1)

class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return self.data.shape[0]
def news_collate_fn(arr):
    arr = torch.LongTensor(arr)
    return arr

news_dataset = NewsDataset(news_combined)
news_dataloader = DataLoader(news_dataset,
                            batch_size=args.batch_size * 4,
                            num_workers=args.num_workers,
                            collate_fn=news_collate_fn)

news_scoring = []
with torch.no_grad():
    for input_ids in tqdm(news_dataloader):
        if args.enable_gpu:
            input_ids = input_ids.cuda()
        news_vec = model.news_encoder(input_ids)
        news_vec = news_vec.to(torch.device("cpu")).detach().numpy()
        news_scoring.extend(news_vec)

news_scoring = np.array(news_scoring)    

dev_dataloader = DataLoaderTest(
        news_index=news_index,
        news_scoring=news_scoring,
        word_dict=word_dict,
        data_dir=valid_dir,
        filename_pat=args.filename_pat,
        args=args,
        worker_size=hvd_size,
        worker_rank=hvd_rank,
        cuda_device_idx=hvd_local_rank,
        enable_prefetch=True,
        enable_shuffle=True,
        enable_gpu=args.enable_gpu,
    )
     


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
42416it [00:06, 6952.22it/s]
100%|██████████| 332/332 [00:34<00:00,  9.68it/s]


In [ ]:
from NRMS.metrics import roc_auc_score, ndcg_score, mrr_score, ctr_score
import csv
import datetime
import time
start_time = time.time()

AUC, MRR, nDCG5, nDCG10, SCORE = [], [], [], [], []
count = 0
outfile = os.path.join(valid_dir, "prediction_{}_{}.tsv".format(datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S"), hvd_local_rank))


def get_mean(arr):
    return [np.array(i).mean() for i in arr]

def write_score(SCORE, outfile):
    # format the score: ImpressionID [Rank-of-News1,Rank-of-News2,...,Rank-of-NewsN]
    for score in tqdm(SCORE):
        argsort = np.argsort(-score[1])
        ranks = np.empty_like(argsort)
        ranks[argsort] = np.arange(len(score[1]))
        score[1] = (ranks + 1).tolist()

    # save the prediction result
    def write_tsv(score):
        with open(outfile, 'a') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerows(score)
    write_tsv(SCORE)
tqdm_util_eval = tqdm(enumerate(dev_dataloader))
def print_metrics(hvd_local_rank, cnt, x):
    tqdm_util.set_description("[{}] Ed: {}: {}".format(hvd_local_rank, cnt, \
        '\t'.join(["{:0.2f}".format(i * 100) for i in x])))
    
with torch.no_grad():    
    for cnt, (impression_ids, log_vecs, log_mask, news_vecs, news_bias, labels) in tqdm_util_eval:
        #if cnt > 0:
        #    break
        #(log_ids, log_mask, input_ids, targets) = data_minibatch
        count = cnt

        if args.enable_gpu:
            #user_ids = user_ids.cuda(non_blocking=True)
            log_vecs = log_vecs.cuda(non_blocking=True)
            log_mask = log_mask.cuda(non_blocking=True)
        # print(impression_ids)
        # print(log_vecs.size())
        # print(log_mask.size())
        # print(len(news_vecs))
        # print(len(news_bias))
        # print(len(labels))
        user_vecs = model.user_encoder(log_vecs, log_mask).to(torch.device("cpu")).detach().numpy()

        for impression_id, user_vec, news_vec, bias, label in zip(
                impression_ids, user_vecs, news_vecs, news_bias, labels):

            #if label.mean() == 0 or label.mean() == 1:
            #    continue

            score = np.dot(
                news_vec, user_vec
            )

            # label is -1 is for test set and prediction only
            if(np.all(label == -1)):
                SCORE.append([impression_id, score])
                continue

            auc = roc_auc_score(label, score)
            mrr = mrr_score(label, score)
            ndcg5 = ndcg_score(label, score, k=5)
            ndcg10 = ndcg_score(label, score, k=10)

            AUC.append(auc)
            MRR.append(mrr)
            nDCG5.append(ndcg5)
            nDCG10.append(ndcg10)

        if cnt % args.log_steps == 0:
            # print_metrics(hvd_rank, cnt * args.batch_size, [1.0])
            print_metrics(hvd_rank, cnt * args.batch_size, get_mean([AUC, MRR, nDCG5,  nDCG10]))

        if cnt % args.save_steps == 0:
            if len(SCORE) > 0:
                tqdm_util.set_description("[{}] Ed: {}: saving {} lines to {}".format(hvd_local_rank, cnt, len(SCORE), outfile))
                write_score(SCORE, outfile)
                SCORE = []

# stop scoring
logging.info("Stop scoring")
dataloader.join()

# save the last batch of scores
if len(SCORE) > 0:
    logging.info("[{}] Ed: {}: saving {} lines to {}".format(hvd_local_rank, cnt, len(SCORE), outfile))
    write_score(SCORE, outfile)
    SCORE = []

# print and save metrics
logging.info("Print final metrics")
print_metrics(hvd_rank, count * args.batch_size, get_mean([AUC, MRR, nDCG5,  nDCG10]))

logging.info(f"Time taken: {time.time() - start_time}")

[INFO 2021-11-23 10:19:48,398] DataLoader __iter__()
[INFO 2021-11-23 10:19:48,402] shut down pool.
get_files: /content/gdrive/My Drive/MINDsmall_dev, behaviors*.tsv


0it [00:00, ?it/s]

files: ['/content/gdrive/My Drive/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-23 10:19:48,417] worker_rank:0, worker_size:1, shuffle:True, seed:10, directory:/content/gdrive/My Drive/MINDsmall_dev, files:['/content/gdrive/My Drive/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-23 10:19:48,418] visible_devices:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO 2021-11-23 10:19:48,423] [StreamReader] path_len:1, paths: ['/content/gdrive/My Drive/MINDsmall_dev/behaviors.tsv']


2285it [08:00, 14.37it/s]

In [ ]:
#outfile

'/content/gdrive/My Drive/MINDsmall_dev/prediction_20211123101247_0.tsv'

In [ ]:
#score = torch.bmm(news_vec, user_vecs.expand_dims(-1)).squeeze(dim=-1)

AttributeError: ignored

In [ ]:

loss = criterion(score, targets)

In [ ]:
loss.data

tensor(41.4614, device='cuda:0')

## Train the NRMS model

In [ ]:
iterator = MINDIterator

In [ ]:
model = NRMSModel(hparams, iterator, seed=seed)

In [ ]:
_= model.test_iterator.load_data_from_file(valid_news_file, valid_behaviors_file)

In [ ]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_graph',
 '_build_newsencoder',
 '_build_nrms',
 '_build_userencoder',
 '_get_input_label_from_iter',
 '_get_loss',
 '_get_news_feature_from_iter',
 '_get_opt',
 '_get_pred',
 '_get_user_feature_from_iter',
 '_init_embedding',
 'eval',
 'fit',
 'group_labels',
 'hparams',
 'loss',
 'model',
 'news',
 'newsencoder',
 'run_eval',
 'run_fast_eval',
 'run_news',
 'run_slow_eval',
 'run_user',
 'scorer',
 'seed',
 'support_quick_scoring',
 'test_iterator',
 'train',
 'train_iterator',
 'train_optimizer',
 'user',
 'userencoder',
 'word2vec_embedding']

In [ ]:
dir(model.test_iterator)


['ID_spliter',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_convert_data',
 '_convert_news_data',
 '_convert_user_data',
 'batch_size',
 'col_spliter',
 'gen_feed_dict',
 'his_size',
 'init_behaviors',
 'init_news',
 'load_data_from_file',
 'load_dict',
 'load_impression_from_file',
 'load_news_from_file',
 'load_user_from_file',
 'npratio',
 'parser_one_line',
 'title_size',
 'uid2index',
 'word_dict']

In [ ]:
??model.test_iterator.load_data_from_file

In [ ]:
#news_file
model.test_iterator.init_news(valid_news_file)

In [ ]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

3781it [01:42, 36.88it/s]
0it [00:00, ?it/s]


IndexError: ignored

In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

step 19720 , total_loss: 1.3392, data_loss: 1.2761: : 19719it [20:54:59,  3.82s/it]

In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)


586it [00:14, 39.70it/s]
236it [04:17,  1.09s/it]
7538it [00:01, 5112.23it/s]


{'group_auc': 0.608, 'mean_mrr': 0.2684, 'ndcg@5': 0.2872, 'ndcg@10': 0.3606}
CPU times: user 8min 49s, sys: 7.46 s, total: 8min 57s
Wall time: 4min 41s


In [ ]:
sb.glue("res_syn", res_syn)

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

## Output Predcition File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [ ]:
test_news_file = os.path.join(data_path, 'MINDlarge_test', r'news.tsv')
test_behaviors_file = os.path.join(data_path, 'MINDlarge_test', r'behaviors.tsv')


In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(test_news_file, test_behaviors_file)

586it [00:14, 39.24it/s]
236it [04:17,  1.09s/it]
7538it [00:01, 5166.76it/s]


In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

7538it [00:00, 35895.52it/s]


In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/